In [16]:
# Libraries used to train models & manipulate data
import numpy as np
import pandas as pd

### Kaggle import: https://github.com/Kaggle/kaggle-api
# kaggle datasets download -f dump.csv --unzip killbot/linkedin-profiles-and-jobs-data
members = pd.read_csv(r'./dump.csv')
# pos_vecs = pd.read_csv(r'./tfidf_positions.csv', header=None) # NN input data
# Filter for useful entries 
members = members.filter(items=['memberUrn', 'posTitle'])
members.head(3)

,memberUrn,posTitle
0,urn:li:member:10013136,Portfolio Executive
1,urn:li:member:10013136,Solution Delivery Executive
2,urn:li:member:10013136,Project Manager


In [17]:
# Simplify the member id
members['memberUrn'] = members['memberUrn'].apply(lambda x: int(x.split(':')[-1]))

print(members.shape)
members.head(3)

(39537, 2)


,memberUrn,posTitle
0,10013136,Portfolio Executive
1,10013136,Solution Delivery Executive
2,10013136,Project Manager


In [18]:
# Group members by position
members_grouped = members.groupby('posTitle')['memberUrn'].apply(list) # 2817 jobs w/ more than 1 member
# Series -> DataFrame & Reset indices
members_grouped = members_grouped.to_frame()
members_grouped = members_grouped.reset_index()

print(members_grouped.shape)
members_grouped.head(3)

(23907, 2)


,posTitle,memberUrn
0,Badminton Head Coach,[121664157]
1,CRO specialist,[110248053]
2,Consultant for Hays Office Support,[487101656]


In [19]:
#! HORRIBLE MEMORY
ids = members['memberUrn'].unique().tolist()

def multiEncodeMembers(urnList):
    l = [0]*len(ids)

    for urn in urnList:
        if urn:
            index = ids.index(urn)
            l[index] = 1
    return l

# Multi=encode the members
members_grouped['memberUrn'] = members_grouped['memberUrn'].apply(multiEncodeMembers)
members_grouped['posTitle'] = members_grouped['posTitle'].drop_duplicates()

print(members_grouped.shape)
members_grouped.head(3)

(23907, 2)


,posTitle,memberUrn
0,Badminton Head Coach,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CRO specialist,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Consultant for Hays Office Support,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
#! HORRIBLE MEMORY
pos_names = members['posTitle'].unique().tolist()

# Multi=encode the members
pos_encoded = members_grouped['posTitle'].apply(lambda x: pos_names.index(x)).to_frame()
pos_encoded = pos_encoded.rename(columns={'posTitle': 'posEncoded'})
# Group data together
train = pd.concat([members_grouped, pos_encoded], axis=1)
train = train.dropna()

train.to_pickle('w2v_train.pkl')

print(len(ids))
print(len(pos_names))
print(train.shape)
train.head(5)

6853
23907
(23907, 3)


,posTitle,memberUrn,posEncoded
0,Badminton Head Coach,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3035
1,CRO specialist,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1583
2,Consultant for Hays Office Support,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20355
3,Content Manager | Writer | Editor | Strategist,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3737
4,"Executive Manager, Change &, Communications","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9824
